In [1]:
username = ""

""

In [2]:
username = "Fro116"

"Fro116"

In [3]:
import NBInclude: @nbinclude
@nbinclude("RecommendationsBase.ipynb");

[ Info: 20221116 02:34:56 RMSE: 1.2439721
[ Info: 20221116 02:34:56 R2: 0.47248912
[ Info: 20221116 02:34:56 Correlation: 0.68750644
[ Info: 20221116 02:34:57 458 explicit items
[ Info: 20221116 02:34:57 461 implicit items
[ Info: 20221116 02:34:57 40 ptw items


In [32]:
ENV["DATAFRAMES_COLUMNS"] = 300;
ENV["DATAFRAMES_ROWS"] = 300;

In [33]:
first(sort(rec_df |> unrelated_display |> unseen_display, :score), 300)

Row,animeid,uid,title,genres,medium,num_episodes,start_date,ranking,original_ranking,explicit,implicit,ptw,baseline_explicit,baseline_implicit,baseline_ptw,score,seen,related
,Int64,Int64,String,String,String,Int64,Date,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,11061,12623,Hunter x Hunter (2011),"['Action', 'Adventure', 'Fantasy', 'Shounen']",tv,148,2011-10-02,63.0,16.0,7.55504,0.0147632,1.65854e-10,7.49254,0.00691353,0.00331507,63.0,false,false
2,5941,1042,Cross Game,"['Drama', 'Romance', 'School', 'Shounen', 'Sports', 'Team Sports']",tv,50,2009-04-05,41.0,24.0,7.80573,0.00553448,0.000161054,7.07991,6.51246e-5,0.000159074,41.0,false,false
3,122,10476,Full Moon wo Sagashite,"['Comedy', 'Drama', 'Music', 'Romance', 'Shoujo', 'Supernatural']",tv,52,2002-04-06,55.0,26.0,7.6145,0.00202783,0.000101821,6.30424,8.21263e-5,0.000100569,55.0,false,false
4,35860,4204,Karakai Jouzu no Takagi-san,"['Comedy', 'Iyashikei', 'Romantic Subtext', 'School', 'Shounen']",tv,12,2018-01-08,68.0,39.0,7.4777,0.00313087,0.000732608,6.2618,0.000577924,0.000723598,68.0,false,false
5,40028,4344,Shingeki no Kyojin: The Final Season,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,16,2020-12-07,57.0,41.0,7.82865,0.000757224,0.00173736,7.36579,0.00502298,0.00171599,57.0,false,false
6,949,7849,Top wo Nerae! Gunbuster,"['Action', 'Comedy', 'Drama', 'Mecha', 'Military', 'Sci-Fi', 'Space']",ova,6,1988-10-07,95.0,43.0,7.27676,0.0201498,0.000183877,7.0182,7.28473e-5,0.000181616,95.0,false,false
7,877,14382,Nana,"['Adult Cast', 'Drama', 'Love Polygon', 'Music', 'Romance', 'Shoujo', 'Slice of Life']",tv,47,2006-04-05,71.0,45.0,7.42396,0.00147085,0.00150352,7.08505,0.000657811,0.00148503,71.0,false,false
8,25835,8607,Shirobako,"['Adult Cast', 'Award Winning', 'Comedy', 'Drama', 'Otaku Culture', 'Workplace']",tv,24,2014-10-09,74.0,49.0,7.39395,0.00391833,0.000878417,6.98263,0.00021843,0.000867615,74.0,false,false
9,1486,5361,Kodomo no Omocha (TV),"['Comedy', 'Drama', 'Love Polygon', 'Romance', 'Shoujo', 'Showbiz']",tv,102,1996-04-05,108.0,52.0,7.25851,0.000608707,6.6722e-5,6.53075,4.7609e-5,6.59013e-5,108.0,false,false


## Curated TV Series

In [5]:
first(
    rec_df |>
    tv_display |>
    implicit_display |>
    explicit_display |>
    unrelated_display |>
    unseen_display,
    50,
)

Row,animeid,uid,title,genres,medium,num_episodes,start_date,ranking,original_ranking,explicit,implicit,ptw,baseline_explicit,baseline_implicit,baseline_ptw,score,seen,related
,Int64,Int64,String,String,String,Int64,Date,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,5941,1042,Cross Game,"['Drama', 'Romance', 'School', 'Shounen', 'Sports', 'Team Sports']",tv,50,2009-04-05,41.0,24.0,7.80573,0.00553448,0.000161054,7.07991,6.51246e-5,0.000159074,41.0,false,false
2,38993,3486,Karakai Jouzu no Takagi-san 2,"['Comedy', 'Iyashikei', 'Romantic Subtext', 'School', 'Shounen']",tv,12,2019-07-07,54.0,71.0,7.79256,0.000493648,0.000352125,6.65439,0.000309038,0.000347794,54.0,false,false
3,122,10476,Full Moon wo Sagashite,"['Comedy', 'Drama', 'Music', 'Romance', 'Shoujo', 'Supernatural']",tv,52,2002-04-06,55.0,26.0,7.6145,0.00202783,0.000101821,6.30424,8.21263e-5,0.000100569,55.0,false,false
4,11061,12623,Hunter x Hunter (2011),"['Action', 'Adventure', 'Fantasy', 'Shounen']",tv,148,2011-10-02,63.0,16.0,7.55504,0.0147632,1.65854e-10,7.49254,0.00691353,0.00331507,63.0,false,false
5,35860,4204,Karakai Jouzu no Takagi-san,"['Comedy', 'Iyashikei', 'Romantic Subtext', 'School', 'Shounen']",tv,12,2018-01-08,68.0,39.0,7.4777,0.00313087,0.000732608,6.2618,0.000577924,0.000723598,68.0,false,false
6,877,14382,Nana,"['Adult Cast', 'Drama', 'Love Polygon', 'Music', 'Romance', 'Shoujo', 'Slice of Life']",tv,47,2006-04-05,71.0,45.0,7.42396,0.00147085,0.00150352,7.08505,0.000657811,0.00148503,71.0,false,false
7,25835,8607,Shirobako,"['Adult Cast', 'Award Winning', 'Comedy', 'Drama', 'Otaku Culture', 'Workplace']",tv,24,2014-10-09,74.0,49.0,7.39395,0.00391833,0.000878417,6.98263,0.00021843,0.000867615,74.0,false,false
8,47904,3261,Hololive Alternative,['Music'],ona,2,2021-05-21,78.0,796.0,7.60826,1.59473e-5,7.24637e-6,6.90516,9.08322e-6,7.15709e-6,78.0,false,false
9,135,514,Hikaru no Go,"['Comedy', 'Drama', 'School', 'Shounen', 'Strategy Game', 'Supernatural']",tv,75,2001-10-10,84.0,53.0,7.41629,0.000439195,0.000123362,6.64984,9.36319e-5,0.000121844,84.0,false,false


## Popular Series

In [7]:
first(
    rec_df |>
    tv_display |>
    inv_implicit_display |>
    explicit_display |>
    unrelated_display |>
    unseen_display,
    10,
)

Row,animeid,uid,title,genres,medium,num_episodes,start_date,ranking,original_ranking,explicit,implicit,ptw,baseline_explicit,baseline_implicit,baseline_ptw,score,seen,related
,Int64,Int64,String,String,String,Int64,Date,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,38524,7051,Shingeki no Kyojin Season 3 Part 2,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,10,2019-04-29,47.0,530.0,7.99369,2.34067e-5,0.00103011,7.63457,0.00531447,0.00101745,47.0,false,false
2,40028,4344,Shingeki no Kyojin: The Final Season,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,16,2020-12-07,57.0,41.0,7.82865,0.000757224,0.00173736,7.36579,0.00502298,0.00171599,57.0,false,false
3,48583,15202,Shingeki no Kyojin: The Final Season Part 2,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,12,2022-01-10,75.0,264.0,7.66423,0.000101164,0.00279556,7.37968,0.00228412,0.00276118,75.0,false,false
4,25777,14441,Shingeki no Kyojin Season 2,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,12,2017-04-01,97.0,58.0,7.35567,0.00171005,0.00123915,7.10045,0.00620125,0.00122392,97.0,false,false
5,35760,1113,Shingeki no Kyojin Season 3,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,12,2018-07-23,113.0,156.0,7.37487,0.000334203,0.00116185,7.23002,0.00539513,0.00114757,113.0,false,false
6,48702,20177,Dance Dance Danseur,"['Drama', 'Performing Arts', 'Seinen']",tv,11,2022-04-09,229.0,758.0,6.94136,4.26823e-5,0.000163432,6.61075,5.84086e-5,0.000161422,229.0,false,false
7,1519,8651,Black Lagoon: The Second Barrage,"['Action', 'Adult Cast', 'Organized Crime', 'Seinen']",tv,12,2006-10-03,231.0,385.0,6.87879,0.000384893,0.000328176,6.83355,0.000517655,0.00032414,231.0,false,false
8,136,3369,Hunter x Hunter,"['Action', 'Adventure', 'Fantasy', 'Shounen']",tv,62,1999-10-16,240.0,908.0,6.94467,0.000119763,0.000858429,6.92017,0.000748238,0.000847872,240.0,false,false
9,46352,854,Blue Period,"['Drama', 'School', 'Seinen', 'Visual Arts']",tv,12,2021-10-02,300.0,476.0,6.75017,0.000315945,0.000857872,6.47542,0.000548962,0.000847322,300.0,false,false


## Highly Rated Series

In [8]:
first(
    rec_df |>
    tv_display |>
    implicit_display |>
    inv_explicit_display |>
    unrelated_display |>
    unseen_display,
    10,
)

Row,animeid,uid,title,genres,medium,num_episodes,start_date,ranking,original_ranking,explicit,implicit,ptw,baseline_explicit,baseline_implicit,baseline_ptw,score,seen,related
,Int64,Int64,String,String,String,Int64,Date,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,46102,11833,Odd Taxi,"['Anthropomorphic', 'Award Winning', 'Mystery']",tv,13,2021-04-06,143.0,98.0,7.03862,0.00274365,1.65854e-10,7.5529,0.000449152,0.000884121,143.0,false,false
2,38474,3041,Yuru Camp△ Season 2,"['CGDCT', 'Iyashikei', 'Slice of Life']",tv,13,2021-01-07,192.0,149.0,7.00477,0.00128907,0.000421757,7.23603,0.000157483,0.000416571,192.0,false,false
3,1453,15078,Maison Ikkoku,"['Adult Cast', 'Comedy', 'Drama', 'Romance', 'Seinen', 'Slice of Life']",tv,96,1986-03-26,204.0,113.0,6.84014,0.00105961,0.000129832,6.95665,2.96269e-5,0.000128235,204.0,false,false
4,37779,397,Yakusoku no Neverland,"['Mystery', 'Psychological', 'Sci-Fi', 'Shounen', 'Survival', 'Suspense']",tv,12,2019-01-10,207.0,115.0,6.77385,0.00523103,0.00193418,7.10592,0.00433655,0.00191039,207.0,false,false
5,34798,2097,Yuru Camp△,"['CGDCT', 'Iyashikei', 'Slice of Life']",tv,12,2018-01-04,218.0,122.0,6.73296,0.00504652,1.65854e-10,6.94816,0.000407609,0.000828574,218.0,false,false
6,6505,5849,There She Is!!,"['Anthropomorphic', 'Comedy', 'Romance']",ona,5,2003-04-20,219.0,136.0,6.86567,0.00142518,1.48429e-5,6.89402,1.51639e-5,1.46602e-5,219.0,false,false
7,31988,7578,Hibike! Euphonium 2,"['Drama', 'Music', 'Performing Arts', 'School']",tv,13,2016-10-06,230.0,145.0,6.91459,0.000729579,0.000239279,7.10218,0.00016961,0.000236336,230.0,false,false
8,2164,10473,Dennou Coil,"['Adventure', 'Award Winning', 'Comedy', 'Drama', 'Mystery', 'Sci-Fi']",tv,26,2007-05-12,235.0,63.0,6.68558,0.0122143,1.65854e-10,6.90604,6.11984e-5,0.00028117,235.0,false,false
9,57,12860,Beck,"['Comedy', 'Drama', 'Music', 'Shounen', 'Slice of Life']",tv,26,2004-10-07,237.0,138.0,6.73106,0.00167111,0.000606465,6.93765,0.000233098,0.000599007,237.0,false,false


## Continue Watching TV Series

In [9]:
first(rec_df |> tv_display |> related_display |> unseen_display, 10)

Row,animeid,uid,title,genres,medium,num_episodes,start_date,ranking,original_ranking,explicit,implicit,ptw,baseline_explicit,baseline_implicit,baseline_ptw,score,seen,related
,Int64,Int64,String,String,String,Int64,Date,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,43608,9080,Kaguya-sama wa Kokurasetai: Ultra Romantic,"['Comedy', 'Psychological', 'Romance', 'School', 'Seinen']",tv,13,2022-04-09,61.0,68.0,7.77993,0.000221997,0.00207485,7.56943,0.001259,0.00204933,61.0,false,true
2,5114,11084,Fullmetal Alchemist: Brotherhood,"['Action', 'Adventure', 'Drama', 'Fantasy', 'Military', 'Shounen']",tv,64,2009-04-05,73.0,38.0,7.42123,0.00428384,0.0046825,7.56178,0.00704877,0.00462492,73.0,false,true
3,21,5123,One Piece,"['Action', 'Adventure', 'Fantasy', 'Shounen']",tv,0,1999-10-20,99.0,157.0,7.46688,0.000169032,0.00215825,7.22565,0.00567394,0.00213171,99.0,false,true
4,19111,14917,Love Live! School Idol Project 2nd Season,"['Idols (Female)', 'Music', 'School', 'Slice of Life']",tv,13,2014-04-06,111.0,40.0,7.27265,0.00151191,0.0001496,6.33955,0.000269016,0.00014776,111.0,false,true
5,93,4625,Mobile Suit Gundam SEED,"['Action', 'Award Winning', 'Drama', 'Mecha', 'Military', 'Romance', 'Sci-Fi', 'Space']",tv,50,2002-10-05,120.0,536.0,7.31981,3.948e-5,0.000102078,6.21977,0.000154477,0.000100823,120.0,false,true
6,16049,21446,Toaru Kagaku no Railgun S,"['Action', 'Sci-Fi', 'Super Power']",tv,24,2013-04-12,124.0,57.0,7.2462,0.00141168,0.000335797,6.62252,0.000325912,0.000331668,124.0,false,true
7,2508,12733,Genshiken 2,"['Comedy', 'Parody', 'Slice of Life']",tv,12,2007-10-10,154.0,97.0,7.0594,0.000300778,4.9008e-5,6.43132,4.55786e-5,4.84051e-5,154.0,false,true
8,37999,22558,Kaguya-sama wa Kokurasetai: Tensai-tachi no Renai Zunousen,"['Comedy', 'Psychological', 'Romantic Subtext', 'School', 'Seinen']",tv,12,2019-01-12,171.0,102.0,6.86324,0.00733794,0.00188058,7.01549,0.00291378,0.00185745,171.0,false,true
9,16706,11671,Kami nomi zo Shiru Sekai: Megami-hen,"['Comedy', 'Harem', 'Otaku Culture', 'Romance', 'Shounen', 'Supernatural']",tv,12,2013-07-09,178.0,118.0,6.97446,0.00107327,0.000202081,6.49511,0.000241001,0.000199596,178.0,false,true


## Curated Specials

In [10]:
first(
    rec_df |>
    inv_tv_display |>
    implicit_display |>
    explicit_display |>
    unrelated_display |>
    unseen_display,
    10,
)

Row,animeid,uid,title,genres,medium,num_episodes,start_date,ranking,original_ranking,explicit,implicit,ptw,baseline_explicit,baseline_implicit,baseline_ptw,score,seen,related
,Int64,Int64,String,String,String,Int64,Date,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,2454,792,Kimagure Orange☆Road: Ano Hi ni Kaeritai,"['Comedy', 'Drama', 'Romance', 'Shounen', 'Slice of Life']",movie,1,1988-10-01,67.0,291.0,7.61675,0.000103949,5.06312e-6,6.64403,4.41065e-6,5.00069e-6,67.0,false,false
2,1376,14879,Ojamajo Doremi Na-i-sho,"['Comedy', 'Fantasy', 'Shoujo']",ova,13,2004-06-26,81.0,609.0,7.52707,4.25445e-5,6.19946e-6,6.52317,7.53166e-6,6.12305e-6,81.0,false,false
3,15989,11310,Saki Achiga-hen: Episode of Side-A Specials,"['School', 'Slice of Life', 'Strategy Game']",special,4,2013-02-20,83.0,124.0,7.49394,0.000228397,4.68813e-6,6.2812,6.87626e-6,4.63031e-6,83.0,false,false
4,9930,2247,Snow Halation,"['Idols (Female)', 'Music']",music,1,2010-12-22,89.0,824.0,7.51546,2.23809e-5,4.24628e-6,6.60559,1.57355e-5,4.1939e-6,89.0,false,false
5,949,7849,Top wo Nerae! Gunbuster,"['Action', 'Comedy', 'Drama', 'Mecha', 'Military', 'Sci-Fi', 'Space']",ova,6,1988-10-07,95.0,43.0,7.27676,0.0201498,0.000183877,7.0182,7.28473e-5,0.000181616,95.0,false,false
6,35472,2350,Yuuki Yuuna wa Yuushabu Shozoku 2,"['Comedy', 'Fantasy', 'Slice of Life']",movie,1,2017-04-15,114.0,161.0,7.45817,2.89078e-5,2.30985e-6,5.45513,9.88166e-7,2.28128e-6,114.0,false,false
7,38422,18344,High Score Girl: Extra Stage,"['Comedy', 'Love Polygon', 'Romantic Subtext', 'School', 'Seinen', 'Video Game']",ova,3,2019-03-20,123.0,86.0,7.19109,0.00296873,3.82398e-5,6.6966,5.82452e-5,3.77694e-5,123.0,false,false
8,35473,9698,Yuuki Yuuna wa Yuushabu Shozoku 3,"['Comedy', 'Fantasy', 'Slice of Life']",movie,1,2017-07-08,127.0,158.0,7.43215,3.92452e-5,1.94934e-6,5.50395,9.17228e-7,1.9252e-6,127.0,false,false
9,39585,8280,Majo Minarai wo Sagashite,"['Award Winning', 'Comedy', 'Drama', 'Shoujo']",movie,1,2020-11-13,130.0,219.0,7.2857,0.000140153,2.43246e-5,6.69204,2.72425e-6,2.40253e-5,130.0,false,false


## Continue Watching Specials

In [11]:
first(rec_df |> inv_tv_display |> related_display |> unseen_display, 10)

Row,animeid,uid,title,genres,medium,num_episodes,start_date,ranking,original_ranking,explicit,implicit,ptw,baseline_explicit,baseline_implicit,baseline_ptw,score,seen,related
,Int64,Int64,String,String,String,Int64,Date,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool,Bool
1,11979,22239,Mahou Shoujo Madoka★Magica Movie 2: Eien no Monogatari,"['Drama', 'Mahou Shoujo', 'Psychological', 'Suspense']",movie,1,2012-10-13,5.0,1.0,9.09651,0.00189216,0.000181213,7.07117,0.000150149,0.000178985,5.0,false,true
2,11977,10202,Mahou Shoujo Madoka★Magica Movie 1: Hajimari no Monogatari,"['Drama', 'Mahou Shoujo', 'Psychological', 'Suspense']",movie,1,2012-10-06,6.0,3.0,8.91963,0.00186193,0.000174886,6.92052,0.0001492,0.000172735,6.0,false,true
3,34376,12370,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 3 - Yakusoku,"['Drama', 'Fantasy', 'Mahou Shoujo', 'Slice of Life']",movie,1,2017-07-08,14.0,5.0,8.69376,0.000966419,1.46603e-5,6.83187,4.82837e-6,1.44799e-5,14.0,false,true
4,48411,4456,Aria the Benedizione,"['Fantasy', 'Iyashikei', 'Sci-Fi', 'Shounen', 'Slice of Life']",movie,1,2021-12-03,21.0,27.0,8.63897,1.19695e-5,1.65854e-10,7.51495,8.97105e-7,1.60904e-5,21.0,false,true
5,34375,19750,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 2 - Tamashii,"['Drama', 'Fantasy', 'Mahou Shoujo', 'Slice of Life']",movie,1,2017-04-15,22.0,9.0,8.4572,0.000714441,1.27284e-5,6.68027,5.00694e-6,1.25717e-5,22.0,false,true
6,31,3322,Neon Genesis Evangelion: Death & Rebirth,"['Drama', 'Mecha', 'Psychological', 'Sci-Fi']",movie,1,1997-03-15,31.0,10.0,8.1019,0.000641813,0.000199152,6.18512,0.000279723,0.000196703,31.0,false,true
7,34374,15364,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou 1 - Tomodachi,"['Drama', 'Fantasy', 'Mahou Shoujo', 'Slice of Life']",movie,1,2017-03-18,35.0,12.0,8.15509,0.000870366,1.48679e-5,6.37926,5.1242e-6,1.46849e-5,35.0,false,true
8,399,6063,Seikai no Senki Special,"['Action', 'Military', 'Romance', 'Sci-Fi', 'Space']",special,1,2001-07-04,36.0,50.0,8.1485,1.76089e-5,1.9562e-6,6.2279,8.92202e-7,1.93198e-6,36.0,false,true
9,6862,20096,K-On!: Live House!,"['CGDCT', 'Comedy', 'Music', 'School']",special,1,2010-01-19,56.0,25.0,7.62416,0.00059253,7.23575e-5,6.47979,0.000141328,7.14675e-5,56.0,false,true
